In [ ]:
#1. Import libraries

import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re

In [ ]:
#2. Configurations

limit = 10 # Set an integer (e.g., 10) to limit number of fighters for testing, or None for no limit.
base_url = "http://ufcstats.com/statistics/fighters" # Base URL for UFC fighters statistics

# Processed Fighters List (to avoid duplicates)
processed_fighter_ids = set()

# Data Collection to store scraped data
fighters_data = []
fights_data = []
events_data = []

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# Corrected function
def get_soup(url, max_retries=3): # Changed parameter name to match logic below
    # Defined headers dictionary
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

    for attempt in range(max_retries):
        try:
            # specific timeout ensures it doesn't hang indefinitely
            response = requests.get(url, headers=headers, timeout=10) 
            
            if response.status_code == 200:
                print(f"Success: Retrieved {url}")
                return BeautifulSoup(response.content, 'html.parser')
            else:
                print(f"Warning: Status code {response.status_code} for {url}")
                time.sleep(2)
                
        except Exception as e:
            print(f"Error fetching {url}: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying... ({attempt + 1}/{max_retries})")
                time.sleep(2)
                
    return None

In [ ]:
# Define the URL you want to test
test_url = "http://ufcstats.com/statistics/fighters"

# 1. Call the function
soup = get_soup(test_url)

# 2. Verify the output
if soup:
    print("\n--- TEST PASSED ---")
    print(f"Object Type: {type(soup)}")
    
    # 3. Check specific content to ensure it's the right page
    # The title tag is usually the best quick check
    page_title = soup.title.text.strip() if soup.title else "No Title Found"
    print(f"Page Title: '{page_title}'")
    
    # 4. (Optional) Print the first 500 characters of HTML to see structure
    print("\n--- HTML PREVIEW (First 500 chars) ---")
    print(soup.prettify()[:500])
else:
    print("\n--- TEST FAILED ---")
    print("The function returned None.")

In [ ]:
#3. Helper Functions 

def clean_height(height_str):
    """Convert height from feet-inches format to total inches."""
    if not height_str or height_str == 'N/A':
        return None
    match = re.match(r"(\d+)' (\d+)\"", height_str)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2))
        return feet * 12 + inches
    return None

def clean_weight(weight_str):
    """Convert weight from string to integer pounds."""
    if not weight_str or weight_str == 'N/A':
        return None
    try:
        return int(weight_str.replace(' lbs.', '').strip())
    except ValueError:
        return None
    
def clean_reach(reach_str):
    """Convert reach from string to integer inches."""
    if not reach_str or reach_str == 'N/A':
        return None
    try:
        return int(reach_str.replace(' in.', '').strip())
    except ValueError:
        return None
    
def clean_time_to_seconds(time_str):
    """Convert time from MM:SS format to total seconds."""
    if not time_str or time_str == 'N/A':
        return None
    try:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds
    except ValueError:
        return None    